### moduls install

In [1]:
!pip install tensorflow==2.9.0
!pip install "numpy<2.0"
!pip install matplotlib seaborn scikit-learn opencv-python-headless

### import moduls

In [2]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

2025-04-14 13:05:36.027281: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### ⭐️ dir setting

In [3]:
model_dir = '/home/nanopore/바탕화면/hajung/watermelon-disease/results/efficientnet_augmented_250414' # ⭐️ 여기 경로 확인
structure_path = os.path.join(model_dir, 'model_structure.json')
weights_path = os.path.join(model_dir, 'model_weights.h5')
config_path = os.path.join(model_dir, 'config.json')
class_info_path = os.path.join(model_dir, 'class_info.json')

### model loading

In [4]:
# class info
def load_class_info(class_info_path):
    # class_info.json 로드
    with open(os.path.join(model_dir, 'class_info.json')) as f:
        class_info = json.load(f)  

    # 클래스 인덱스 → 라벨 딕셔너리 생성
    idx_to_class = {entry['index']: entry['class'] for entry in class_info}
    
    # 입력 이미지 크기
    img_size = (class_info[0]['height'], class_info[0]['width'])
    
    return idx_to_class, img_size

In [5]:

# 모델 + 클래스 정보 불러오기 (추론 전용)
def load_inference_model(model_dir):
    # 모델 구조 + 가중치
    with open(os.path.join(model_dir, 'model_structure.json')) as f:
        model = model_from_json(f.read())
    model.load_weights(os.path.join(model_dir, 'model_weights.h5'))

    return model


### input data

In [6]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
import matplotlib.pyplot as plt
import numpy as np

In [7]:

# 전처리 함수
def preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=img_size)    
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array) # 정규화
    img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가
    
    return img_array

### run

In [8]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [9]:
# path setting
structure_path = 'best_model/model_structure.json'
weights_path = 'best_model/model_weights.h5'
config_path = 'best_model/config.json'
class_names_path = 'best_model/class_names.json'


In [10]:
# 후처리 함수
def decode_prediction(preds, idx_to_class):
    pred_index = np.argmax(preds, axis=1)[0]
    pred_class = idx_to_class[pred_index]
    confidence = float(preds[0][pred_index])
    return pred_class, confidence

# 이미지 확인
def image_print(img):
    plt.imshow(img[0])
    plt.axis('off')
    plt.show()
    return img

# 예측 함수
def predict_image(model, img, idx_to_class):
    preds = model(img)
    pred_class, confidence = decode_prediction(preds, idx_to_class)
    return pred_class, confidence

#### ⭐️ model activate

In [12]:
# call variations
idx_to_class, img_size = load_class_info('경로/파일이름')
img_path = '/home/nanopore/바탕화면/hajung/input/건전.jpg'  # ⭐️ 예측할 이미지 경로
img = preprocess_image(img_path, img_size)
model = load_inference_model(model_dir)

# run model
pred_class, confidence = predict_image(model, img, idx_to_class)

print(structure_path)
print(f"📌 예측 클래스: {pred_class} ({confidence * 100:.2f}%)")

2025-04-14 13:07:01.084436: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-04-14 13:07:01.084468: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nanopore-desktop): /proc/driver/nvidia/version does not exist
2025-04-14 13:07:01.084773: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


best_model/model_structure.json
📌 예측 클래스: Mosaic_Virus (95.53%)
